In [1]:
using QuantumOptics

# Parameters
Nc = 16
γ = 1.
g = γ/2.
κ = 0.5γ
ωr = .15γ
Δc = -γ
Δa = -2γ
η = γ
tmax = 600
tsteps = 10*tmax
dt = tmax/tsteps
tlist = [0:dt:tmax;]

# Hilbert space
bc = FockBasis(Nc)
ba = SpinBasis(1//2)

# Operators
a = destroy(bc) ⊗ one(ba)
ad = dagger(a)
sm = one(bc) ⊗ sigmam(ba)
sp = dagger(sm)

Operator(dim=34x34)
  basis: [Fock(cutoff=16) ⊗ Spin(1/2)]
adjoint(sparse([18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], ComplexF64[1.0+0.0im, 1.0+0.0im, 1.0+0.0im, 1.0+0.0im, 1.0+0.0im, 1.0+0.0im, 1.0+0.0im, 1.0+0.0im, 1.0+0.0im, 1.0+0.0im, 1.0+0.0im, 1.0+0.0im, 1.0+0.0im, 1.0+0.0im, 1.0+0.0im, 1.0+0.0im, 1.0+0.0im], 34, 34))

In [2]:
# Hamiltonian
H0 = -Δc*ad*a - Δa*sp*sm + η*(a + ad)
Hx = g*(a*sp + ad*sm) # ∝ cos(x)

# Jump operators
J = [sqrt(2κ)*a, sqrt(2γ)*sm]
Jdagger = map(dagger, J)

2-element Vector{Operator{CompositeBasis{Vector{Int64}, Tuple{FockBasis{Int64}, SpinBasis{1//2, Int64}}}, CompositeBasis{Vector{Int64}, Tuple{FockBasis{Int64}, SpinBasis{1//2, Int64}}}, LinearAlgebra.Adjoint{ComplexF64, SparseArrays.SparseMatrixCSC{ComplexF64, Int64}}}}:
 Operator(dim=34x34)
  basis: [Fock(cutoff=16) ⊗ Spin(1/2)]
adjoint(sparse([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  24, 25, 26, 27, 28, 29, 30, 31, 32, 33], [2, 3, 4, 5, 6, 7, 8, 9, 10, 11  …  25, 26, 27, 28, 29, 30, 31, 32, 33, 34], ComplexF64[1.0+0.0im, 1.41421+0.0im, 1.73205+0.0im, 2.0+0.0im, 2.23607+0.0im, 2.44949+0.0im, 2.64575+0.0im, 2.82843+0.0im, 3.0+0.0im, 3.16228+0.0im  …  2.64575+0.0im, 2.82843+0.0im, 3.0+0.0im, 3.16228+0.0im, 3.31662+0.0im, 3.4641+0.0im, 3.60555+0.0im, 3.74166+0.0im, 3.87298+0.0im, 4.0+0.0im], 34, 34))
 Operator(dim=34x34)
  basis: [Fock(cutoff=16) ⊗ Spin(1/2)]
adjoint(sparse([18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 

In [3]:
function fquantum(t, psi, u) # psi is the quantum, u the classical part
  x = u[1]
  return H0 + Hx*cos(x), J, Jdagger
end

adsm = ad*sm # Define to avoid doing a multiplication at every step
function fclassical!(du, u, psi, t) # du is a vector containing the increments of the classical variables
  du[1] = 2ωr*u[2]
  du[2] = 2g*sin(u[1])*real(expect(adsm, psi))
end

fclassical! (generic function with 1 method)

In [4]:
tout, ρt = semiclassical.master_dynamic(tlist, ψsc0, fquantum, fclassical!)

LoadError: UndefVarError: `ψsc0` not defined